In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.plotting.register_matplotlib_converters()

import numpy as np
import seaborn as sns
import missingno as mgn

import plotly.express as px

Reading the data with standard preprocessing steps

In [ ]:
# Just the new file only - for now
df = pd.read_excel('../data/discover_2024-03-26.xlsx', sheet_name='data')
df.columns = [col.lower() for col in df.columns]
## add more steps (renaming, etc.)?

In [ ]:
df.columns

### Article (content) versions

We want to label each version.
* Version changes when there is a new `publication date`
* Version changes when there is a new `word count`
* Version does NOT change with a change in `URL`
* Version does NOT change with a change in the `date` column

In [ ]:
# Only the necessary columns
# we still need the 'date' column for imputation
df_cnt = df[['page_efahrer_id', 'date', 'published_at', 'word_count']]
df_cnt = df_cnt.sort_values(['page_efahrer_id', 'date', 'published_at'])
df_cnt

In [ ]:
mgn.matrix(df_cnt);

There are some columns where the publication date changed but the `word count` was not updated!

In [ ]:
wcna_idx = df_cnt[df_cnt.word_count.isna() & df_cnt.published_at.notna()].index
wcna_idx

NOT the other way around:

In [ ]:
df_cnt[df_cnt.word_count.notna() & df_cnt.published_at.isna()].index

The best assumption that it did not change (significantly??), so still forward-fill it.

In [ ]:
df_cnt = df_cnt.ffill()

In [ ]:
sus = df_cnt.loc[wcna_idx]

#### Versions

In [ ]:
df_cnt['publ_at_enc'] = df_cnt.groupby('page_efahrer_id')['published_at'].transform(lambda x: pd.factorize(x)[0])
df_cnt

How many versions does each article have?

In [ ]:
to_plot = df_cnt[['page_efahrer_id', 'publ_at_enc']].groupby('page_efahrer_id').max()#.reset_index()
first_publ_date = df_cnt[['page_efahrer_id', 'published_at']].groupby('page_efahrer_id').min()
first_publ_date = first_publ_date.rename({'published_at': 'First publication date'}, axis=1)
to_plot = to_plot.join(first_publ_date)
to_plot = to_plot.rename({'publ_at_enc': 'Number of versions'}, axis=1)
to_plot

In [ ]:
px.scatter(data_frame=to_plot, x='Number of versions', y='First publication date')

Article with 61 (!!!) versions:

In [ ]:
metrics_cols = ['page_canonical_url', 'daily_likes', 
               'daily_dislikes', 'impressions', 'video_play', 
               'discover_clicks', 'discover_impressions']

In [ ]:
article105259 = df[df['page_efahrer_id']==105259].sort_values(['date', 'page_canonical_url'])
article_first_url = article105259[metrics_cols + ['date']].drop_duplicates(subset=['date'], keep='first')

In [ ]:
import matplotlib.pyplot as plt

xticks=pd.date_range(df.date.min(), df.date.max(), freq='2M')
fig = article_first_url.plot(kind='bar', x='date', y=metrics_cols, subplots=True, figsize=(6, 12), 
                       xticks=xticks)
plt.gca().set_xticklabels([x.strftime('%a\n%d\n%h\n%Y') for x in xticks]);
#plt.xticks(ticks=df[['impressions', 'published_at']].resample('W', on='published_at').max().index);
#plt.xticks(ticks=pd.date_range(df.date.min(), df.date.max(), freq='2M'),
#           labels=pd.date_range(df.date.min(), df.date.max(), freq='2M'));